In [2]:
import requests 
from bs4 import BeautifulSoup 
import csv 
from datetime import date
from datetime import datetime
import numpy as np

# Retrieve Data from Intentionalist: Takeout and Delivery Directory
# URL: https://intentionalist.com/covid-19-takeout-delivery-directory/
## Author: Jeffrey Lai

In [3]:
URL = "https://intentionalist.com/covid-19-takeout-delivery-directory/"
r = requests.get(URL)

In [4]:
soup = BeautifulSoup(r.content, 'html5lib') 

In [5]:
table = soup.findAll('div', attrs = {'class':'b-name'})

# Build Restaurant Name List

In [48]:
rest = []
for i in table:
    #rest.appendtable[3].text
    rest.append(i.text)
rest = list(map(lambda st : str.replace(st,"+","\+"),rest))

# Build Restaurant Status List

In [33]:
status = []
st = soup.findAll('div', attrs = {'class':'b-status'})
for i in st:
    status.append(i.text)

# Build Restaurant Service List

In [34]:
serv = []
service = (soup.findAll('div', attrs = {'class':'b-services'}))
for i in service:
    serv.append(i.text[8:100])

In [35]:
import re
t = soup.find('div', text = re.compile('Good Egg'),attrs = {'class':'b-name'}).find_next_siblings("div")

# Retrieve Restaurant Location

In [37]:
arr = []
rest2 = []
for store in rest:
    rest2.append(store.replace("\\",""))
    t = soup.find('div', text = re.compile(store),attrs = {'class':'b-name'})#.find_next_siblings("div")
    if t is None :
        arr.append("NaN")
    elif t.find_next_siblings("div")[1].find(attrs = {'class':'b-label'}).text == "Neighborhood" and len(t.find_next_siblings("div")) == 2:
        arr.append(t.find_next_siblings("div")[1].find(attrs = {'class':'b-label'}).next_sibling) # some restaurants only have neighborhood
    elif t.find_next_siblings("div")[1].find(attrs = {'class':'b-label'}).text == "Neighborhood" and len(t.find_next_siblings("div")) > 2:
        arr.append(t.find_next_siblings("div")[2].find(attrs = {'class':'b-label'}).next_sibling)
    else:
        t2 = t.find_next_siblings("div")
        arr.append(t2[1].find(attrs = {'class':'b-label'}).next_sibling)
    


# Combine All Restaurant Information into Data Frame

In [38]:
import pandas as pd
df = pd.DataFrame(arr,rest2)


In [39]:
df['status'] = status

In [40]:
df['service'] = serv

In [41]:
df = df.reset_index().rename(columns={'index':'name', 0:"address"})

In [42]:
df[df['name'].str.contains("Hong Kong")]

,name,address,status,service
292,Hong Kong Bistro,"507 Maynard Ave S #511C, Seattle, WA 98104",,Takeout
783,A + Hong Kong Kitchen,"419 6th Ave S, Seattle, WA 98104",Temporarily Closed,N/A


# Extract Data for restaurants in Washington State only

In [43]:
df["address"] = df['address'].str.strip()
df["state"] = df['address'].str.extract('(WA|Seattle|([9][8][0-9]+)$)')[0]
#df[df['state'].isnull()]

In [44]:
df["state"] = np.where(df["name"].isin([ "Shiro's Sushi"]),"WA",df["state"])

# Write Data To local Hard Drive 

In [45]:
df.to_csv('C:\\Users\\jeffr\\takeout_{}.csv'.format(datetime.strftime(date.today(),'%Y%m%d')),encoding='utf-8-sig')